In [1]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import requests
import pandas as pd
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime
import urllib.request
import dash_table
import dash_table.FormatTemplate as FormatTemplate
from dash_table.Format import Sign
import numpy as np
import ssl
import json
import time

In [2]:
def get_n_top_coins(n=10):
    url = 'https://api.lunarcrush.com/v2?data=market&key=39o5gbbtx7n1gxkvnhf6na&limit={}&sort=acr&type=fast'.format(n)
    response = requests.get(url).json()
    df = pd.json_normalize(response['data'])
    df.rename(columns={
        's': 'Symbol',
        'n': 'Name',
        'p': 'Price',
        'v': 'Volume',
        'pc': '24 Hours',
        'pch': '1 Hour',
        'mc': 'Market Cap',
        'gs': 'Galaxy Score',
        'ss': 'Social Score',
        'acr': 'ALTRank',
        't':'Number of tweets',
        'sd':'Social Dominance',
        'd' : 'Market Dominance'
        }, inplace=True)
    df['24 Hours'] = df['24 Hours'].div(100).round(4)
    df['1 Hour'] = df['1 Hour'].div(100).round(4)

    return df[['ALTRank','Galaxy Score','Social Score','Symbol','Name','Price','Volume','24 Hours','1 Hour','Market Cap','Number of tweets','Social Dominance','Market Dominance']]


In [3]:
def get_market_table(n=50):
    market_data = get_n_top_coins(n)
    market_table = dash_table.DataTable(
        id='market_table',
        data=market_data.to_dict('records'),
        columns=[
            {'id': 'ALTRank', 'name': 'ALTRank', 'type': 'numeric'},
            {'id': 'Galaxy Score', 'name': 'Galaxy Score (PT)',
             'type': 'numeric'},
            {'id': 'Social Score', 'name': 'Social Score (PT)',
             'type': 'numeric'},
            {'id': 'Symbol', 'name': 'Symbol', 'type': 'text'},
            {'id': 'Name', 'name': 'Name', 'type': 'text'},
            {
                'id': 'Price',
                'name': 'Price',
                'type': 'numeric',
                'format': FormatTemplate.money(6),
                },
            {
                'id': 'Volume',
                'name': 'Volume',
                'type': 'numeric',
                'format': FormatTemplate.money(2),
                },
            {
                'id': '24 Hours',
                'name': '24 Hours',
                'type': 'numeric',
                'format': FormatTemplate.percentage(2).sign(Sign.positive),
                },
            {
                'id': '1 Hour',
                'name': '1 Hour',
                'type': 'numeric',
                'format': FormatTemplate.percentage(2).sign(Sign.positive),
                },
            {
                'id': 'Market Cap',
                'name': 'Market Cap',
                'type': 'numeric',
                'format': FormatTemplate.money(0),
                },
            ],
        style_data_conditional=[
    #         data_bars(market_data, 'Volume')[0],
            {'if': {'filter_query': '{24 Hours} > 0',
             'column_id': '24 Hours'}, 'color': 'green',
             'fontWeight': 'bold'},
            {'if': {'filter_query': '{24 Hours} < 0',
             'column_id': '24 Hours'}, 'color': 'red', 'fontWeight': 'bold'
             },
            {'if': {'filter_query': '{24 Hours} = 0',
             'column_id': '24 Hours'}, 'color': 'blue', 'fontWeight': 'bold'
             },
            {'if': {'filter_query': '{1 Hour} > 0', 'column_id': '1 Hour'},
             'color': 'green', 'fontWeight': 'bold'},
            {'if': {'filter_query': '{1 Hour} < 0', 'column_id': '1 Hour'},
             'color': 'red', 'fontWeight': 'bold'},
            {'if': {'filter_query': '{1 Hour} = 0', 'column_id': '1 Hour'},
             'color': 'blue', 'fontWeight': 'bold'},
            {'if': {'row_index': 'odd'},
             'backgroundColor': 'rgb(248, 248, 248)'},
            ],
        style_cell_conditional=[{'if': {'column_id': 'Name'},
                                'textAlign': 'left'}, 
    #                             {
    #         'if': {'column_id': 'Volume'},
    #         'width': '200px',
    #         'color': 'blue',
    #         'minWidth': '200px',
    #         'maxWidth': '200px',
    #         'overflow': 'hidden',
    #         'textOverflow': 'ellipsis',}, 
                                {'if': {'column_id': 'Symbol'}, 'fontWeight': 'bold',
                'textAlign': 'center'}],
        style_header={'backgroundColor': '#DCDCDC', 'fontWeight': 'bold'},
        # jakiś error podczas sortowania np 24 hours lub 1houur
        sort_action='custom',
        editable=False,
        style_as_list_view=True,
        page_size=5,
    #     fill_width=False
        )
    return market_table

In [16]:
coins = ['BTC','DOT','ETH','DOGE','LTC','ADA','XRP']

In [18]:
def get_coin_data(symbol,data_points=365,interval='day'):
    url = "https://api.lunarcrush.com/v2?data=assets&key=39o5gbbtx7n1gxkvnhf6na&symbol={}&data_points={}&interval={}".format(symbol,data_points,interval)
    response = requests.get(url).json()
    data = response['data'][0]
    df = pd.json_normalize(data['timeSeries'])
    df['time'] = pd.to_datetime(df['time'],unit='s')
    data = pd.json_normalize(data)
    details = data[['name','symbol','price_btc','market_cap','percent_change_24h','percent_change_7d','percent_change_30d','volume_24h','max_supply']]

    return (df,details.to_dict(orient='records')[0])

In [55]:
def build_header():
    return html.Div([
        html.Div([ 
            html.H4("Our Title", className = 'app-header-title'),
            html.P("Some descriptons, maybe our names or somethings like this", 'app-header-subtitle'),
        ], className = 'app-header-main'),
        html.Img(
            src = 'assets/crunchips.png',
            className = 'logo'
        ),
        html.Audio(id = 'easter-egg', controls=True, autoPlay = True, hidden = True)
    ], className = 'app-header')

In [5]:
def build_tabs():
    return html.Div(
        id = 'tabs',
        className="tabs",
        children = [
            dcc.Tabs(
                id = 'control-tabs',
                value = 'tab1',
                className="custom-tabs",
                children=[
                    dcc.Tab(
                        id = 'Main-tab',
                        label = 'Main Tab',
                        value = "tab1",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id = 'Sub-tab',
                        label = 'Sub Tab',
                        value = "tab2",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id = 'Rest-tab',
                        label = 'Rest Tab',
                        value = "tab3",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                ],
            )
        ],
    )

In [45]:
def build_tab_1():
    return [
        html.Div([
            html.Div([
                get_market_table(),
            ], className = 'tabble-wrapper'),
            
            html.Div([
                html.Div([
                    html.P("Type:"),
                    dcc.Dropdown(
                        id='types', 
                        className = 'dropdown-type',
                        value='Market Dominance', 
                        options=[{'label': x, 'value': x} 
                                 for x in ['Market Dominance', 'Social Dominance', 'Number of tweets']],
                        clearable=False, 
                        style={'color': '#212121'} 
                    ),
                    html.P("Top N coins:"),
                    dcc.Dropdown(
                        id='top_n', 
                        className = 'dropdown-top',
                        value='5', 
                        options=[{'value': x, 'label': x} 
                                 for x in ['3','5','10','15']],
                        clearable=False, 
                        style=
                                    { 'width': '135px',
                                      'color': '#212121',
                                    } 
                    ),
                ], className = 'dropdowns-wrapper'), 
                html.Div([
                    dcc.Graph(id="pie-chart")
                ], className = 'gauge-wrapper'),

                html.Div([
                    dcc.Graph(id="scatter-chart")
                ], className = 'scatter-wrapper')
            ], className = 'bottom-wrapper')
            
        ], className = 'first-div')
    ]

In [46]:
def build_tab_2():
    return [
        html.Div([
            html.Div([
                html.Div([
                    html.Label("Coin"),
                    dcc.Dropdown(
                          id='coin-dropdown',
                          clearable=False,
                          value='BTC',
                          options=[{'label': c, 'value': c} for c in coins],
                            style={'color': '#212121'})
                ],className = 'dropdowns-wrapper'),
                html.Div([
                    dcc.Graph(id='graph')
                ], className = 'plotline-wrapper')
                
            ], className = 'top-wrapper')
        ], className = 'second-div')
    ]

In [26]:
def build_tab_3():
    return []

In [57]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        build_header(),
        build_tabs(),
        html.Div(id = 'app-content')
    ], style = {'padding': 20})
    
])
@app.callback(
    Output('app-content', 'children'),
    Input('control-tabs', 'value')
)
def prepare_top(tab_switch):
    if tab_switch == 'tab1':
        return build_tab_1()
    elif tab_switch == 'tab2':
        return build_tab_2()
    else:
        return build_tab_3()

@app.callback(
    [Output("pie-chart", "figure"),
     Output("scatter-chart","figure")], 
    [Input("types", "value"),
     Input("top_n","value")])
def generate_chart(types,top_n):
    top_n = int(top_n)
    df = get_n_top_coins(top_n)
    fig_pie = px.pie(df, values=df[types],names=df['Symbol'],hole=.5,title=types)
    
    fig_pie.update_layout(plot_bgcolor = '#082255', paper_bgcolor = '#082255', font_color = '#FFFFFF', width = 400)
    
    fig_scatter = px.scatter(df, x=df['Price'], y=df['Volume'],
                 size=df[types], color=df['Symbol'],
                 hover_name=df['Symbol'], log_y=True,log_x=True, size_max=100,)
    fig_scatter.update_layout(title=types,
                             xaxis_title="Price ($)",
                             yaxis_title="Volume($)",)
    
    fig_scatter.update_layout( paper_bgcolor = '#082255', font_color = '#FFFFFF')
    
    return fig_pie,fig_scatter
@app.callback(
    Output('easter-egg', 'src'),
    Input('coin-dropdown', 'value')
)
def easteregg(coin):
    if coin =='DOGE':
        return 'assets/dog.wav'
    else:
        return ''
@app.callback(
    Output('graph', 'figure'), 
    [Input('coin-dropdown', 'value')]
)
def update_figure(coin):
    x, details = get_coin_data(coin)
    x['MA-Low'] = x.low.rolling(window=20).mean()
    x['MA-High'] = x.high.rolling(window=20).mean()
    fig = go.Figure(data=[go.Candlestick(x=x['time'],
                                         open=x['open'],
                                         high=x['high'],
                                         low=x['low'],
                                         close=x['close'],
                                         name='{} price'.format(details['symbol']))])
    
    fig.update_layout(title=details['name'],
                      yaxis_tickformat='$',
                      xaxis_title='Date',
                      yaxis_title='Price')
    
    fig.update_xaxes(title_text='Date',
                     rangeslider_visible=True,
                     rangeselector=dict(buttons=list([dict(count=1,
                                                           label='1M',
                                                           step='month',
                                                           stepmode='backward'),
                                                      dict(count=6,
                                                           label='6M',
                                                           step='month',
                                                           stepmode='backward'),
                                                      dict(count=1,
                                                           label='YTD',
                                                           step='year', 
                                                           stepmode='todate'),
                                                      dict(count=1,
                                                           label='1Y', 
                                                           step='year', 
                                                           stepmode='backward'),
                                                      dict(step='all')]), bgcolor='black'))
    
    fig.add_trace(go.Scatter(x=x['time'],
                             y=x['MA-Low'],
                             mode='lines',
                             name='Low-Moving Average',
                             line={'color':'#ff9100',
                                  'width':1}))
    fig.add_trace(go.Scatter(x=x['time'],
                             y=x['MA-High'],
                             mode='lines',
                             name='High-Moving Average',
                             line={'color':'#006eff',
                                  'width':1}))
    
    fig.update_layout(xaxis_rangeslider_visible=True)
    fig.update_layout( paper_bgcolor = '#082255', font_color = '#FFFFFF', width = 1100)
    return fig

app.run_server(mode='external', port = 8054)

Dash app running on http://127.0.0.1:8054/
aa
